In [1]:
import requests
import os
import re
import sys
import json
import requests
import argparse
import time
import codecs
from bs4 import BeautifulSoup
from six import u

In [2]:
def fetch(url):
    response = requests.get(url)
    response = requests.get(url, cookies={'over18': '1'})  # 一直向 server 回答滿 18 歲了 !
    return response


In [3]:
widths = [
        (126,    1), (159,    0), (687,     1), (710,   0), (711,   1),
        (727,    0), (733,    1), (879,     0), (1154,  1), (1161,  0),
        (4347,   1), (4447,   2), (7467,    1), (7521,  0), (8369,  1),
        (8426,   0), (9000,   1), (9002,    2), (11021, 1), (12350, 2),
        (12351,  1), (12438,  2), (12442,   0), (19893, 2), (19967, 1),
        (55203,  2), (63743,  1), (64106,   2), (65039, 1), (65059, 0),
        (65131,  2), (65279,  1), (65376,   2), (65500, 1), (65510, 2),
        (120831, 1), (262141, 2), (1114109, 1),
]


def calc_len(string):
    def chr_width(o):
        global widths
        if o == 0xe or o == 0xf:
            return 0
        for num, wid in widths:
            if o <= num:
                return wid
        return 1
    return sum(chr_width(ord(c)) for c in string)


def pretty_print(push, title, date, author, link):
    pattern = '%3s\t%s%s%s\t%s'
    padding = ' ' * (50 - calc_len(title))
    print(pattern % (push, title, padding, date, author, link))

In [4]:
def parse_article_meta(ent):
    ''' Step-3 (revised): parse the metadata in article entry '''
    # 基本要素都還在
    meta = {
        'title': ent.find('div.title', first=True).text,
        'push': ent.find('div.nrec', first=True).text,
        'date': ent.find('div.date', first=True).text
    }

    try:
        # 正常狀況取得資料
        meta['author'] = ent.find('div.author', first=True).text
        meta['link'] = ent.find('div.title > a', first=True).attrs['href']

    except AttributeError:
        # 但碰上文章被刪除時，就沒有辦法像原本的方法取得 作者 跟 連結
        if '(本文已被刪除)' in meta['title']:
            # e.g., "(本文已被刪除) [haudai]"
            match_author = re.search('\[(\w*)\]', meta['title'])
            match_link = re.search('\[(\w*)\]', meta['title']) 
            if match_author:
                meta['author'] = match_author.group(1)
            if match_link:
                meta['link'] = match_link.group(1)
        elif re.search('已被\w*刪除', meta['title']):
            # e.g., "(已被cappa刪除) <edisonchu> op"
            match_author = re.search('\<(\w*)\>', meta['title'])
            if match_author:
                meta['author'] = match_author.group(1)
        elif re.search('-',meta['author']):
            meta['link'] = '-'
    return meta

    # 最終仍回傳統一的 dict() 形式 paired data
    #return meta

In [5]:
from requests_html import HTML

def parse_article_entries(doc):
    html = HTML(html=doc)
    post_entries = html.find('div.r-ent')
    return post_entries

In [6]:
url = 'https://www.ptt.cc/bbs/Gossiping/index.html'
resp = fetch(url)  # step-1
post_entries = parse_article_entries(resp.text)  # step-2

for entry in post_entries:
    meta = parse_article_meta(entry)
    pretty_print(meta['push'], meta['title'], meta['date'], meta['author'])  

    # pretty_print(meta['push'], meta['title'], meta['date'], meta['author'])  # for below results

TypeError: pretty_print() missing 1 required positional argument: 'link'

In [12]:
import re
import urllib.parse
## from utils import pretty_print

def get_metadata_from(url):

    def parse_next_link(doc):
        domain = 'https://www.ptt.cc/' 
        html = HTML(html=doc)
        controls = html.find('.action-bar a.btn.wide')
        link = controls[1].attrs.get('href')
        return urllib.parse.urljoin(domain, link)

    resp = fetch(url)
    post_entries = parse_article_entries(resp.text)
    next_link = parse_next_link(resp.text)

    metadata = [parse_article_meta(entry) for entry in post_entries]
    return metadata, next_link

def get_paged_meta(url, num_pages):
    collected_meta = []
    domain = 'https://www.ptt.cc/'
    
    for _ in range(num_pages):
        posts, link = get_metadata_from(url)
        collected_meta += posts

        url = urllib.parse.urljoin(domain, link)
    return collected_meta

#metadata = get_paged_meta(start_url, num_pages=10)
#for meta in metadata:
    #pretty_print(meta['push'], meta['title'], meta['date'], meta['author'],meta['link'])

In [7]:
def get_posts(metadata):
    # 將所有文章連結收集並串接成完整 URL
    post_links = [
        urllib.parse.urljoin(domain, meta['link'])
        for meta in metadata if 'link' in meta
    ]
    with Pool(processes=8) as pool:
        contents = pool.map(fetch, post_links)
        return contents
    

In [8]:
import time
from multiprocessing import Pool

domain = 'https://www.ptt.cc/'

In [9]:
def parse_article_content(doc,href):
    print(href)
    if '.html' in (href):
        html = HTML(html=doc)
        post_content = html.find('#main-content', first=True).text
        soup = BeautifulSoup(doc, 'html.parser')
        main_content = soup.find(id="main-content")
        metas = main_content.select('div.article-metaline')
        if metas:
            author = metas[0].select('span.article-meta-value')[0].string if metas[0].select('span.article-meta-value')[0] else author
            title = metas[1].select('span.article-meta-value')[0].string if metas[1].select('span.article-meta-value')[0] else title
            date = metas[2].select('span.article-meta-value')[0].string if metas[2].select('span.article-meta-value')[0] else date
            # remove meta nodes
            for meta in metas:
                meta.extract()
            for meta in main_content.select('div.article-metaline-right'):
                meta.extract()
        PTT_URL = 'https://www.ptt.cc'
        # ex. link would be <a href="/bbs/PublicServan/M.1127742013.A.240.html">Re: [問題] 職等</a>
        link = PTT_URL + href
        article_id = re.sub('\.html', '', href.split('/')[-1])        
        pushes = main_content.find_all('div', class_='push')

        for push in pushes:
            push.extract() ## 移除指定該變數
        filtered = [ v for v in main_content.stripped_strings if v[0] not in [u'※', u'◆'] and v[:2] not in [u'--'] ]
        expr = re.compile(u(r'[^\u4e00-\u9fa5\u3002\uff1b\uff0c\uff1a\u201c\u201d\uff08\uff09\u3001\uff1f\u300a\u300b\s\w:/-_.?~%()]'))
        for i in range(len(filtered)):
            filtered[i] = re.sub(expr, '', filtered[i])

        filtered = [_f for _f in filtered if _f]  # remove empty strings
        filtered = [x for x in filtered if article_id not in x]  # remove last line containing the url of the article
        content = ' '.join(filtered)
        content = re.sub(r'(\s)+', ' ', content)
        meta = {
            'content':content,
            'article_id':article_id
        }
        return meta
    else:
        meta = {
            'content':'被刪除',
            'article_id': '無'            
        }
        return meta

In [13]:
if __name__ == '__main__':
    pages = 20
    start_url = 'https://www.ptt.cc/bbs/movie/index.html'
    start = time.time()

    metadata = get_paged_meta(start_url, num_pages=pages)
    resps = get_posts(metadata)
    
    contents = [parse_article_content(content.text, metadata['link']) for content, metadata in zip(resps, metadata)]

    print('花費: %f 秒' % (time.time() - start))

    print('共%d項結果：' % len(resps))
    ##for post, resps, text in zip(metadata, resps,contents):
    ##    print('{0} {1: <15} {2}, 網頁內容共 {3} 字, {4}'.format(
    ##        post['date'], post['author'], post['title'], len(resps.text), text['content']))

/bbs/movie/M.1537508611.A.491.html
/bbs/movie/M.1537509959.A.90E.html
/bbs/movie/M.1537513464.A.E81.html
/bbs/movie/M.1537513819.A.35D.html
tontontonni
/bbs/movie/M.1537514655.A.A1F.html
/bbs/movie/M.1537515619.A.5A6.html
/bbs/movie/M.1535458622.A.135.html
/bbs/movie/M.1535458800.A.A5B.html
/bbs/movie/M.1535805002.A.1A4.html
/bbs/movie/M.1537456810.A.847.html
/bbs/movie/M.1537456987.A.395.html
/bbs/movie/M.1537457828.A.9DD.html
/bbs/movie/M.1537459328.A.667.html
/bbs/movie/M.1537460903.A.6BD.html
/bbs/movie/M.1537462436.A.3C3.html
/bbs/movie/M.1537465897.A.121.html
/bbs/movie/M.1537471551.A.560.html
/bbs/movie/M.1537475393.A.2C6.html
/bbs/movie/M.1537475493.A.9EB.html
/bbs/movie/M.1537484794.A.1BF.html
/bbs/movie/M.1537491372.A.ED2.html
/bbs/movie/M.1537498331.A.B64.html
/bbs/movie/M.1537498790.A.B17.html
/bbs/movie/M.1537499273.A.DD1.html
/bbs/movie/M.1537503192.A.155.html
/bbs/movie/M.1537503744.A.BB8.html
/bbs/movie/M.1537505063.A.738.html
/bbs/movie/M.1537505211.A.3AF.html
/bbs/mov

/bbs/movie/M.1537096759.A.5D5.html
/bbs/movie/M.1537100203.A.F1D.html
/bbs/movie/M.1537102227.A.3B7.html
/bbs/movie/M.1537102362.A.52D.html
tieamonk
/bbs/movie/M.1537071526.A.C78.html
/bbs/movie/M.1537072049.A.6C3.html
a122239
/bbs/movie/M.1537074097.A.664.html
/bbs/movie/M.1537074649.A.B52.html
/bbs/movie/M.1537076677.A.A8D.html
/bbs/movie/M.1537076749.A.6FF.html
/bbs/movie/M.1537077725.A.B5E.html
patrickchang
/bbs/movie/M.1537080169.A.95E.html
/bbs/movie/M.1537080222.A.770.html
/bbs/movie/M.1537082309.A.419.html
/bbs/movie/M.1537082931.A.EB3.html
ckckcck5577
/bbs/movie/M.1537084353.A.456.html
/bbs/movie/M.1537084819.A.D49.html
/bbs/movie/M.1537084950.A.7AB.html
qpr322
/bbs/movie/M.1537085966.A.0F1.html
/bbs/movie/M.1537043806.A.8FC.html
/bbs/movie/M.1537049018.A.70D.html
/bbs/movie/M.1537049175.A.772.html
/bbs/movie/M.1537050800.A.CF7.html
/bbs/movie/M.1537052583.A.01F.html
/bbs/movie/M.1537055704.A.22E.html
/bbs/movie/M.1537055782.A.296.html
/bbs/movie/M.1537056881.A.FAF.html
/bbs/m

In [11]:
contents[3]

NameError: name 'contents' is not defined

In [15]:
zip(metadata , contents)

NameError: name 'metadata' is not defined

In [76]:
z

{'title': '[選片] 命帶追逐 引爆點 小偷家族',
 'push': '5',
 'date': '9/03',
 'author': 'stepaside',
 'link': '/bbs/movie/M.1535958298.A.313.html',
 'content': '作者 stepaside (紐牛) 看板 movie 標題 [選片] 命帶追逐 引爆點 小偷家族 時間 Mon Sep 3 15:04:56 2018 手機排版，跪求不要噓我嗚嗚 三部都是近期內有些心動的片 不知道大家比較推哪一部呢？ 身邊較多人看過小偷家族， 似乎比較沉重呢。 命帶追逐預告片看起來ㄎㄧㄤㄎㄧㄤ的， 不曉得會不會其實內容不有趣。 引爆點是衝著吳慷仁的演技， 可是版上滿多負評的。 休假想出門看電影， 請求各位推薦了~~~~ 感謝感謝 Sent from JPTT on my Samsung SMJ730GM. ',
 'article_id': 'M.1535958298.A.313',
 '_id': ObjectId('5b8cf14e360a3e15d49824f3')}

In [122]:
filename = './crawl_file/'+ test['article_id']+'.json'
with open(filename, 'w') as outfile:
    json.dump(test, outfile, ensure_ascii=False)

### 匯入MongoDB

In [18]:
import pymongo
from pymongo import MongoClient


In [19]:
client = pymongo.MongoClient(host='localhost', port=27017)

In [20]:
db = client.ptt

### show database

In [21]:
if __name__ == '__main__':
    client = pymongo.MongoClient("localhost", 27017, maxPoolSize=50)
    d = dict((db, [collection for collection in client[db].collection_names()])
             for db in client.database_names())
    print(json.dumps(d))

{"admin": ["system.version"], "config": ["system.sessions"], "local": ["startup_log"]}


/Users/aaron/Documents/anaconda3/envs/crawler/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: database_names is deprecated. Use list_database_names instead.
  after removing the cwd from sys.path.
/Users/aaron/Documents/anaconda3/envs/crawler/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  after removing the cwd from sys.path.


In [22]:
#db.article.drop()
collection = db['article']

In [42]:
result = collection.find({}, {'title':1})
print(type(result))
print(result[0])

<class 'pymongo.cursor.Cursor'>
{'_id': ObjectId('5b8cedb0360a3e15d49824d9'), 'title': '[選片] 命帶追逐 引爆點 小偷家族'}


In [23]:
for x, y in zip(metadata, contents):
    z = {**x, **y} 
    result = collection.insert([z])    
    #filename = './crawl_file/'+ z['article_id']+'.json'
    #with open(filename, 'w') as outfile:
        #json.dump(z, outfile, ensure_ascii=False)

/Users/aaron/Documents/anaconda3/envs/crawler/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [24]:
result = collection.find({}, {'title':1, '_id':0})
print(type(result))


<class 'pymongo.cursor.Cursor'>


In [25]:
for i in result:
    print(i)

{'title': 'Re: [討論] 劉德華主演警匪片經典《五億探長雷洛傳》上映27週年'}
{'title': '[討論] 布麗拉森哪一部影片的扮裝最讓你驚豔'}
{'title': '[好雷] 我很好，讓死亡證明自己活著'}
{'title': '[討論] 威秀寶寶是不是接到置入了?'}
{'title': '(本文已被刪除) [tontontonni]'}
{'title': 'Re: [討論] 劉德華主演警匪片經典《五億探長雷洛傳》上映27週年'}
{'title': '[好雷] 《我很好》 無彩青春的輓歌'}
{'title': '[版規] 電影版版規 201808'}
{'title': '[公告] 電影版板規修訂說明'}
{'title': '[公告] 板規修訂 板規參議規範'}
{'title': '[超級好雷] 藍色項圈'}
{'title': '[好雷] 《裸愛殺機》'}
{'title': '[ 普雷] 鬼修女'}
{'title': '[新聞] 賈樟柯《在清朝》2019年開機'}
{'title': '[討論] 與神同行快下檔了'}
{'title': '[討論] 老人家看終極戰士(無雷)'}
{'title': '[新聞] 粽邪 票房近5000萬 海外版權9國爭搶'}
{'title': '[片單] 校園青少女魯蛇'}
{'title': '[討論] 投名狀和讓子彈飛哪一部更為經典？'}
{'title': '[請益] 已看舊版惡魔島值得花錢再最新版嗎??'}
{'title': '[新聞] 黑木瞳昔日母女變情敵 《退》戰小三廣末'}
{'title': '[問片] 一部有關批踢踢的微電影 (有雷 (已解'}
{'title': '[問片] 請問跟「攝影」相關的電影'}
{'title': '[新聞] 狂穿褲裝被取笑 布蕾克萊佛莉「神反擊」'}
{'title': '[情報] 《猛禽小隊》演員候選人名單曝光'}
{'title': '[好雷] 《一屍到底》，電影的重新定義。'}
{'title': '[選片] 惡魔島VS終極戰士'}
{'title': '[討論] 睡著也好醒來也罷'}
{'title': '[片單] 看完影評後整個劇情反轉的電影'}
{'title': '[好雷] 血薄荷—硬派媽媽的復仇之路'}
{'title': '[

In [52]:
client.drop_database('article')
